In [14]:
import json
import random
import yaml
import logging
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from mesa import Agent, Model
from mesa.agent import AgentSet

# Add utils to path
sys.path.insert(0, str(Path.cwd()))

# Import config loader and logger
from utils.config import load_config, get_config
from utils.logger import setup_logging

# Load configuration
config = load_config("config.yaml")
logger = setup_logging(
    log_level=config.simulation.get("log_level", "INFO"),
    log_file=config.logging_config.get("log_file")
)

logger.info("=" * 60)
logger.info(f"Agricultural Agent-Based Simulation")
logger.info(f"Agents: {config.simulation.get('n_agents')}")
logger.info(f"Timesteps: {config.simulation.get('n_timesteps')}")
logger.info(f"Random Seed: {config.simulation.get('random_seed')}")
logger.info("=" * 60)

# Set random seed for reproducibility
random.seed(config.simulation.get("random_seed", 42))

# Load data files from config
data_paths = config.data_paths
with open(data_paths["zones_file"]) as f:
    zones_data = json.load(f)

with open(data_paths["crops_file"]) as f:
    crops_data = json.load(f)

with open(data_paths["climate_file"]) as f:
    climate_data = json.load(f)

with open(data_paths["soil_moisture_file"]) as f:
    soil_moisture = json.load(f)

cfg = get_config()
n_agents = cfg.simulation.get("n_agents", 100)
n_timesteps = cfg.simulation.get("n_timesteps", 10)

logger.info(f"Loaded {len(zones_data['zones'])} zones")
logger.info(f"Loaded {len(crops_data['crops'])} crops")
logger.info(f"Loaded climate profiles for all zones")
logger.info(f"Loaded soil moisture data")

2026-02-14 20:09:06,834 - agroai_sim - INFO - Logging to logs\simulation.log
2026-02-14 20:09:06,837 - agroai_sim - INFO - ============================================================
2026-02-14 20:09:06,837 - agroai_sim - INFO - Agricultural Agent-Based Simulation
2026-02-14 20:09:06,838 - agroai_sim - INFO - Agents: 200
2026-02-14 20:09:06,839 - agroai_sim - INFO - Timesteps: 10
2026-02-14 20:09:06,842 - agroai_sim - INFO - Random Seed: 42
2026-02-14 20:09:06,844 - agroai_sim - INFO - ============================================================
2026-02-14 20:09:06,863 - agroai_sim - INFO - Loaded 6 zones
2026-02-14 20:09:06,863 - agroai_sim - INFO - Loaded 24 crops
2026-02-14 20:09:06,867 - agroai_sim - INFO - Loaded climate profiles for all zones
2026-02-14 20:09:06,868 - agroai_sim - INFO - Loaded soil moisture data


In [15]:
class Zone:
    def __init__(self, zone_id, data, climate_profile, soil_moisture):
        self.id = zone_id
        self.name = data["name"]
        self.soil_types = data["soil_types"]
        self.dominant_soil = data["dominant_soil"]
        self.viable_crops = data["viable_crops"]
        self.agriculture_type = data["agriculture_type"]
        self.climate_profile = climate_profile
        self.water_price = data.get("water_price", 0.4)

        # Soil moisture correctly extracted here
        sm_data = soil_moisture["zones"][zone_id]
        self.mean_sm = sm_data["mean"]
        self.std_sm = sm_data["std"]

        self.shared_crop_yield = {}
        self.shared_crop_profit = {}  # Track profit too for smarter decisions

        # Environmental state per timestep (set by FarmModel.step)
        self.is_drought_year = False
        self.is_flood_year = False
        self.pest_pressure = 0.0          # 0-1 scale
        self.price_shock_factor = 1.0     # multiplier on market prices
        self.timestep = 0

In [16]:
class FarmerAgent(Agent):
    def __init__(self, model, unique_id, zone, land_size, strategy_type):
        super().__init__(model)
        self.unique_id = unique_id
        self.zone = zone
        self.land_size = land_size
        self.strategy_type = strategy_type
        self.chosen_crop = None
        self.yield_t_ha = 0.0
        self.revenue_mad = 0.0
        self.cost_mad = 0.0
        self.profit_mad = 0.0
        self.profit = 0.0          # kept for backward compat
        self.memory = []

    def step(self):
        self.decide_crop()
        self.compute_yield()
        self.compute_profit()

    # ------------------------------------------------------------------
    # CROP DECISION
    # ------------------------------------------------------------------
    def decide_crop(self):
        cfg = get_config()
        available_crops = list(crops_data["crops"].keys())
        suitability_scores = {}
        for crop in available_crops:
            suitability_scores[crop] = self.compute_suitability(crop)

        if self.strategy_type == "SHARED":
            shared_wt = cfg.learning.get("shared_weight", 0.6)
            community_yield = self.zone.shared_crop_yield
            community_profit = self.zone.shared_crop_profit
            source = community_profit if community_profit else community_yield
            if source:
                max_val = max(source.values()) if source else 1.0
                for crop in available_crops:
                    if crop in source and max_val > 0:
                        community_bonus = source[crop] / max_val
                        suitability_scores[crop] += shared_wt * community_bonus
                    else:
                        suitability_scores[crop] *= (1.0 - shared_wt * 0.3)

        self.chosen_crop = max(suitability_scores, key=suitability_scores.get)

    # ------------------------------------------------------------------
    # SUITABILITY SCORING (unchanged logic, cleaner)
    # ------------------------------------------------------------------
    def compute_suitability(self, crop_name):
        cfg = get_config()
        zone_weight = cfg.suitability.get("zone_weight", 0.4)
        soil_weight = cfg.suitability.get("soil_weight", 0.25)
        climate_weight = cfg.suitability.get("climate_weight", 0.25)
        moisture_weight = cfg.suitability.get("moisture_weight", 0.1)

        crop = crops_data["crops"][crop_name]
        ecocrop = crop["ecocrop"]
        score = 0.0
        weight_total = 0.0

        # 1. Zone suitability
        zone_suitable = crop.get("ideal_zones", crop.get("suitable_zones", []))
        if self.zone.id in zone_suitable:
            score += 1.0 * zone_weight
        elif self.zone.id in crop.get("marginal_zones", []):
            score += 0.5 * zone_weight
        weight_total += zone_weight

        # 2. Soil suitability
        preferred_soils = crop.get("ideal_soils", crop.get("preferred_soils", []))
        if self.zone.dominant_soil in preferred_soils:
            score += 1.0 * soil_weight
        else:
            score += 0.3 * soil_weight
        weight_total += soil_weight

        # 3. Soil Moisture suitability
        moisture_data = soil_moisture.get(self.zone.id, {})
        if moisture_data:
            moisture_score = moisture_data.get("mean_rzsm_normalized", 0.5)
            crop_moisture_pref = crop.get("moisture_preference", "medium")
            if crop_moisture_pref == "high":
                moisture_score = moisture_score
            elif crop_moisture_pref == "low":
                moisture_score = 1.0 - moisture_score
            else:
                moisture_score = 1.0 - abs(moisture_score - 0.5) * 2
            score += moisture_score * moisture_weight
        weight_total += moisture_weight

        # 4. Climate suitability
        climate_profile = self.zone.climate_profile
        temp_score = 0.0
        precip_score = 0.0
        t_opt_min = ecocrop.get("temp_opt_min_C", ecocrop.get("temp_opt_min_c", 15))
        t_opt_max = ecocrop.get("temp_opt_max_C", ecocrop.get("temp_opt_max_c", 30))
        t_abs_min = ecocrop.get("temp_abs_min_C", ecocrop.get("temp_abs_min_c", 5))
        t_abs_max = ecocrop.get("temp_abs_max_C", ecocrop.get("temp_abs_max_c", 40))
        p_opt_min = ecocrop.get("precip_opt_min_mm", 400)
        p_opt_max = ecocrop.get("precip_opt_max_mm", 1200)
        p_abs_min = ecocrop.get("precip_abs_min_mm", 200)
        p_abs_max = ecocrop.get("precip_abs_max_mm", 2000)

        for month_key in [f"month_{m}" for m in range(1, 13)]:
            monthly = climate_profile.get(month_key, {})
            tavg = monthly.get("tavg_mean", 20)
            if t_opt_min <= tavg <= t_opt_max:
                temp_score += 1.0
            elif t_abs_min <= tavg <= t_abs_max:
                temp_score += 0.5

            prectot = monthly.get("prectotcorr", {"mean": 1.5})
            p_mean = prectot["mean"] * 365
            if p_opt_min <= p_mean <= p_opt_max:
                precip_score += 1.0
            elif p_abs_min <= p_mean <= p_abs_max:
                precip_score += 0.5

        temp_score /= 12
        precip_score /= 12
        climate_score = 0.5 * temp_score + 0.5 * precip_score
        score += climate_score * climate_weight
        weight_total += climate_weight

        return score / weight_total if weight_total > 0 else 0.0

    # ------------------------------------------------------------------
    # YIELD with environmental stressors (drought, flood, pests)
    # ------------------------------------------------------------------
    def compute_yield(self):
        cfg = get_config()
        crop_info = crops_data["crops"][self.chosen_crop]
        base_yield = crop_info["base_yield_t_ha"]
        suitability = self.compute_suitability(self.chosen_crop)

        # Base variability
        var_min = cfg.yield_params.get("variability_min", 0.85)
        var_max = cfg.yield_params.get("variability_max", 1.15)
        variability = var_min + (var_max - var_min) * random.random()

        # Environmental stress factors (from zone state set by FarmModel)
        stress_factor = 1.0

        # Drought stress
        if self.zone.is_drought_year:
            water_pref = crop_info.get("water_need", "medium")
            drought_impact = {"very_low": 0.95, "low": 0.85, "medium": 0.65,
                              "high": 0.45, "very_high": 0.30}
            stress_factor *= drought_impact.get(water_pref, 0.65)

        # Flood stress
        if self.zone.is_flood_year:
            water_pref = crop_info.get("water_need", "medium")
            flood_impact = {"very_low": 0.40, "low": 0.55, "medium": 0.70,
                            "high": 0.85, "very_high": 0.90}
            stress_factor *= flood_impact.get(water_pref, 0.70)

        # Pest pressure (0-1 scale; pest_pressure=0.4 → ~10% yield loss)
        pest = self.zone.pest_pressure
        if pest > 0:
            pest_loss = pest * 0.25  # max 25% yield loss at pressure=1.0
            stress_factor *= (1.0 - pest_loss)

        self.yield_t_ha = base_yield * suitability * variability * stress_factor

    # ------------------------------------------------------------------
    # PROFIT = Revenue - Costs  (in MAD = Moroccan Dirham)
    # Uses FAOSTAT prices + dynamic cost model + config finance params
    # ------------------------------------------------------------------
    def compute_profit(self):
        cfg = get_config()
        fin = cfg.finance
        crop_info = crops_data["crops"][self.chosen_crop]
        cost_model = crops_data.get("cost_model", {})

        # ---- REVENUE ----
        price_per_ton = crop_info.get("faostat_price_mad_per_ton", 3000)

        # Price volatility (random ± config.finance.price_volatility)
        volatility = fin.get("price_volatility", 0.15)
        price_var = 1.0 + random.uniform(-volatility, volatility)

        # Market shock (rare price crash / boom)
        shock_prob = fin.get("market_shock_probability", 0.05)
        if random.random() < shock_prob:
            price_var *= random.choice([0.6, 0.7, 1.4, 1.5])  # crash or boom

        # Zone-level price shock from environmental events
        price_var *= self.zone.price_shock_factor

        effective_price = price_per_ton * price_var

        # SHARED cooperative gets price premium (bulk selling, better negotiation)
        if self.strategy_type == "SHARED":
            premium = fin.get("shared_price_premium", 0.20)
            effective_price *= (1.0 + premium)

        # Post-harvest loss
        if self.strategy_type == "SHARED":
            post_harvest_loss = fin.get("shared_post_harvest_loss", 0.05)
        else:
            post_harvest_loss = fin.get("individual_post_harvest_loss", 0.15)

        sellable_yield = self.yield_t_ha * (1.0 - post_harvest_loss)
        self.revenue_mad = sellable_yield * effective_price * self.land_size

        # ---- COSTS ----
        cost_cat = crop_info.get("cost_category", "cereal")
        labor_days = crop_info.get("labor_days_per_ha", 20)
        smag = cost_model.get("smag_mad_per_day", 84.37)
        water_cost_m3 = self.zone.water_price  # zone-specific water price

        # Water need in m³/ha (1 mm = 10 m³/ha)
        water_needs = cost_model.get("water_need_mm", {})
        water_pref = crop_info.get("water_need", "medium")
        water_mm = water_needs.get(water_pref, 500)
        water_m3_per_ha = water_mm * 10

        # Per-hectare cost components
        labor_cost = labor_days * smag
        water_cost = water_m3_per_ha * water_cost_m3
        fertilizer_cost = cost_model.get("fertilizer_npk_mad_per_ha", {}).get(cost_cat, 2000)
        seeds_cost = cost_model.get("seeds_mad_per_ha", {}).get(cost_cat, 800)
        mechanization_cost = cost_model.get("mechanization_mad_per_ha", {}).get(cost_cat, 1000)

        cost_per_ha = labor_cost + water_cost + fertilizer_cost + seeds_cost + mechanization_cost

        # SHARED cooperatives have lower per-unit costs (bulk purchasing, shared equipment)
        if self.strategy_type == "SHARED":
            cost_efficiency = fin.get("shared_cost_efficiency", 0.70)  # 30% savings
        else:
            cost_efficiency = fin.get("individual_cost_efficiency", 0.90)  # 10% savings

        cost_per_ha *= cost_efficiency

        # Input cost inflation over time
        inflation = fin.get("input_cost_inflation", 0.02)
        cost_per_ha *= (1.0 + inflation) ** self.zone.timestep

        self.cost_mad = cost_per_ha * self.land_size
        self.profit_mad = self.revenue_mad - self.cost_mad
        self.profit = self.profit_mad  # backward compat

        self.memory.append({
            "timestep": self.zone.timestep,
            "crop": self.chosen_crop,
            "yield": self.yield_t_ha,
            "revenue_mad": self.revenue_mad,
            "cost_mad": self.cost_mad,
            "profit": self.profit_mad,
            "price_per_ton": effective_price,
            "drought": self.zone.is_drought_year,
            "flood": self.zone.is_flood_year,
        })

In [17]:
class FarmModel(Model):
    def __init__(self, zones_data, climate_data, crops_data, soil_moisture, n_agents):
        super().__init__()

        self.crops_data = crops_data
        self.zones = {}
        self.current_step = 0

        for zone_id, zone_info in zones_data["zones"].items():
            climate_profile = climate_data["zones"][zone_id]["climate_profile"]
            self.zones[zone_id] = Zone(
                zone_id,
                zone_info,
                climate_profile,
                soil_moisture
            )

        # ================================================================
        # PAIRED DESIGN: Equal land split for fair SHARED vs INDIVIDUAL
        # For every SHARED farmer, an INDIVIDUAL farmer gets the EXACT
        # same zone and land size. The ONLY difference is strategy.
        # ================================================================
        cfg = get_config()

        # Ensure even number of agents for perfect pairing
        n_pairs = n_agents // 2
        actual_agents = n_pairs * 2
        if actual_agents != n_agents:
            logger.info(f"  Adjusted agent count from {n_agents} to {actual_agents} for equal pairing")

        agents_list = []
        agent_id = 0

        for i in range(n_pairs):
            # Generate ONE random zone and land size per pair
            zone = random.choice(list(self.zones.values()))
            land_size = random.uniform(1, 10)

            # Create SHARED farmer
            shared_agent = FarmerAgent(
                model=self,
                unique_id=agent_id,
                zone=zone,
                land_size=land_size,
                strategy_type="SHARED"
            )
            agents_list.append(shared_agent)
            agent_id += 1

            # Create INDIVIDUAL farmer with SAME zone & land size
            individual_agent = FarmerAgent(
                model=self,
                unique_id=agent_id,
                zone=zone,
                land_size=land_size,
                strategy_type="INDIVIDUAL"
            )
            agents_list.append(individual_agent)
            agent_id += 1

        self.farmers = AgentSet(agents_list)

        # Log the pairing summary
        shared_count = sum(1 for a in agents_list if a.strategy_type == "SHARED")
        indiv_count = sum(1 for a in agents_list if a.strategy_type == "INDIVIDUAL")
        shared_land = sum(a.land_size for a in agents_list if a.strategy_type == "SHARED")
        indiv_land = sum(a.land_size for a in agents_list if a.strategy_type == "INDIVIDUAL")
        logger.info(f"  Paired design: {shared_count} SHARED + {indiv_count} INDIVIDUAL")
        logger.info(f"  Total land — SHARED: {shared_land:.1f} ha, INDIVIDUAL: {indiv_land:.1f} ha (equal)")

    def step(self):
        self.current_step += 1
        self._generate_environmental_events()
        self.farmers.shuffle_do("step")
        self.update_shared_knowledge()

    def _generate_environmental_events(self):
        """Roll environmental dice per zone per timestep using config params."""
        cfg = get_config()
        yld = cfg.yield_params
        drought_prob = yld.get("drought_years_probability", 0.15)
        flood_prob = yld.get("flood_probability", 0.08)
        pest_baseline = yld.get("pest_pressure_baseline", 0.4)

        for zone in self.zones.values():
            zone.timestep = self.current_step

            # Drought: zone-independent annual roll
            zone.is_drought_year = random.random() < drought_prob

            # Flood: zone-independent annual roll (drought and flood are mutually exclusive)
            if zone.is_drought_year:
                zone.is_flood_year = False
            else:
                zone.is_flood_year = random.random() < flood_prob

            # Pest pressure: baseline + random variation
            zone.pest_pressure = min(1.0, max(0.0,
                pest_baseline + random.gauss(0, 0.15)))

            # Price shock from supply disruption (drought/flood → price spike)
            if zone.is_drought_year:
                zone.price_shock_factor = random.uniform(1.10, 1.35)  # scarcity premium
            elif zone.is_flood_year:
                zone.price_shock_factor = random.uniform(0.85, 1.10)  # mixed
            else:
                zone.price_shock_factor = 1.0

            if zone.is_drought_year or zone.is_flood_year:
                event = "DROUGHT" if zone.is_drought_year else "FLOOD"
                logger.debug(f"  t={self.current_step} Zone {zone.id}: {event} (pest={zone.pest_pressure:.2f})")

    def update_shared_knowledge(self):
        for zone in self.zones.values():
            shared_agents = [
                a for a in self.farmers
                if a.zone == zone and a.strategy_type == "SHARED"
            ]

            if not shared_agents:
                zone.shared_crop_yield = {}
                zone.shared_crop_profit = {}
                continue

            crop_yields = {}
            crop_profits = {}

            for agent in shared_agents:
                crop = agent.chosen_crop
                crop_yields.setdefault(crop, []).append(agent.yield_t_ha)
                crop_profits.setdefault(crop, []).append(agent.profit_mad)

            zone.shared_crop_yield = {
                crop: sum(yields) / len(yields)
                for crop, yields in crop_yields.items()
            }
            zone.shared_crop_profit = {
                crop: sum(profits) / len(profits)
                for crop, profits in crop_profits.items()
            }

In [18]:
import time

# Get parameters from config
n_agents = config.simulation.get("n_agents", 100)
n_timesteps = config.simulation.get("n_timesteps", 30)

logger.info(f"\n{'='*60}")
logger.info(f"Starting simulation: {n_agents} agents × {n_timesteps} timesteps")
logger.info(f"{'='*60}")

# Track execution time
start_time = time.time()

# Initialize model
model = FarmModel(zones_data, climate_data, crops_data, soil_moisture, n_agents)
logger.info(f"✓ Model initialized with {n_agents} farmers (paired design)")
logger.info(f"  Finance params: price_premium={config.finance.get('shared_price_premium')}, "
            f"cost_eff={config.finance.get('shared_cost_efficiency')}, "
            f"post_harvest_loss SHARED={config.finance.get('shared_post_harvest_loss')} "
            f"vs INDIV={config.finance.get('individual_post_harvest_loss')}")

# Track per-timestep aggregate stats
timestep_history = []

# Run simulation
for t in range(n_timesteps):
    model.step()

    # Collect per-timestep snapshot
    shared_yields = [a.yield_t_ha for a in model.farmers if a.strategy_type == "SHARED"]
    indiv_yields = [a.yield_t_ha for a in model.farmers if a.strategy_type == "INDIVIDUAL"]
    shared_profits = [a.profit_mad for a in model.farmers if a.strategy_type == "SHARED"]
    indiv_profits = [a.profit_mad for a in model.farmers if a.strategy_type == "INDIVIDUAL"]

    # Check for any environmental events across zones
    drought_zones = [z.id for z in model.zones.values() if z.is_drought_year]
    flood_zones = [z.id for z in model.zones.values() if z.is_flood_year]

    timestep_history.append({
        "timestep": t + 1,
        "avg_yield_shared": np.mean(shared_yields),
        "avg_yield_individual": np.mean(indiv_yields),
        "avg_profit_shared": np.mean(shared_profits),
        "avg_profit_individual": np.mean(indiv_profits),
        "drought_zones": len(drought_zones),
        "flood_zones": len(flood_zones),
    })

    if t % 5 == 0 or t == n_timesteps - 1:
        events = ""
        if drought_zones:
            events += f" | DROUGHT in {len(drought_zones)} zones"
        if flood_zones:
            events += f" | FLOOD in {len(flood_zones)} zones"
        logger.info(f"  t={t+1:>3}/{n_timesteps}  "
                    f"Yield S={np.mean(shared_yields):.2f} I={np.mean(indiv_yields):.2f}  "
                    f"Profit S={np.mean(shared_profits):>10,.0f} I={np.mean(indiv_profits):>10,.0f} MAD"
                    f"{events}")

elapsed = time.time() - start_time
logger.info(f"✓ Simulation completed in {elapsed:.2f} seconds")

ts_df = pd.DataFrame(timestep_history)

2026-02-14 20:09:07,014 - agroai_sim - INFO - 
2026-02-14 20:09:07,015 - agroai_sim - INFO - Starting simulation: 200 agents × 10 timesteps
2026-02-14 20:09:07,016 - agroai_sim - INFO - ============================================================
2026-02-14 20:09:07,023 - agroai_sim - INFO -   Paired design: 100 SHARED + 100 INDIVIDUAL
2026-02-14 20:09:07,023 - agroai_sim - INFO -   Total land — SHARED: 555.8 ha, INDIVIDUAL: 555.8 ha (equal)
2026-02-14 20:09:07,027 - agroai_sim - INFO - ✓ Model initialized with 200 farmers (paired design)
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encodin

In [19]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy import stats as scipy_stats

try:
    import seaborn as sns
except ImportError:
    sns = None

output_dir = Path("results")
output_dir.mkdir(exist_ok=True)

# ============================================================================
# COLLECT PER-AGENT RESULTS
# ============================================================================
logger.info("\n" + "="*60)
logger.info("ANALYSIS & RESULTS")
logger.info("="*60)

rows = []
agents = getattr(model, 'farmers', None) or getattr(model, 'agents', None) or []
for a in agents:
    mem = getattr(a, 'memory', [])
    if not mem:
        continue
    yields = [m['yield'] for m in mem]
    profits = [m['profit'] for m in mem]
    revenues = [m.get('revenue_mad', 0) for m in mem]
    costs = [m.get('cost_mad', 0) for m in mem]
    drought_count = sum(1 for m in mem if m.get('drought', False))
    flood_count = sum(1 for m in mem if m.get('flood', False))

    # Crop diversity (number of distinct crops chosen over all timesteps)
    crops_chosen = list(set(m['crop'] for m in mem))

    rows.append({
        'unique_id': getattr(a, 'unique_id', None),
        'strategy': getattr(a, 'strategy_type', None),
        'zone': getattr(getattr(a, 'zone', None), 'id', None),
        'land_size_ha': getattr(a, 'land_size', None),
        'mean_yield': np.mean(yields),
        'std_yield': np.std(yields),
        'mean_revenue_mad': np.mean(revenues),
        'mean_cost_mad': np.mean(costs),
        'mean_profit_mad': np.mean(profits),
        'std_profit_mad': np.std(profits),
        'total_profit_mad': sum(profits),
        'profit_per_ha_mad': np.mean(profits) / a.land_size if a.land_size > 0 else 0,
        'droughts_experienced': drought_count,
        'floods_experienced': flood_count,
        'final_crop': mem[-1]['crop'],
        'n_crops_tried': len(crops_chosen),
    })

df = pd.DataFrame(rows)

# ============================================================================
# HEADLINE METRICS
# ============================================================================
shared = df[df['strategy'] == 'SHARED']
indiv = df[df['strategy'] == 'INDIVIDUAL']

metrics = {
    "total_agents": int(len(df)),
    "shared_agents": int(len(shared)),
    "individual_agents": int(len(indiv)),
    "n_timesteps": n_timesteps,
    "currency": "MAD",
    # -- Yield --
    "avg_yield_all": float(df['mean_yield'].mean()),
    "avg_yield_shared": float(shared['mean_yield'].mean()),
    "avg_yield_individual": float(indiv['mean_yield'].mean()),
    "solara_t_ha": float(shared['mean_yield'].mean() - indiv['mean_yield'].mean()),
    # -- Profit (MAD) --
    "avg_profit_shared_mad": float(shared['mean_profit_mad'].mean()),
    "avg_profit_individual_mad": float(indiv['mean_profit_mad'].mean()),
    "profit_advantage_shared_mad": float(shared['mean_profit_mad'].mean() - indiv['mean_profit_mad'].mean()),
    "avg_profit_per_ha_shared": float(shared['profit_per_ha_mad'].mean()),
    "avg_profit_per_ha_individual": float(indiv['profit_per_ha_mad'].mean()),
    # -- Revenue & Costs --
    "avg_revenue_shared_mad": float(shared['mean_revenue_mad'].mean()),
    "avg_revenue_individual_mad": float(indiv['mean_revenue_mad'].mean()),
    "avg_cost_shared_mad": float(shared['mean_cost_mad'].mean()),
    "avg_cost_individual_mad": float(indiv['mean_cost_mad'].mean()),
    # -- Risk --
    "profit_cv_shared": float(shared['std_profit_mad'].mean() / shared['mean_profit_mad'].mean()) if shared['mean_profit_mad'].mean() != 0 else 0,
    "profit_cv_individual": float(indiv['std_profit_mad'].mean() / indiv['mean_profit_mad'].mean()) if indiv['mean_profit_mad'].mean() != 0 else 0,
}

# Statistical test: paired t-test on profit
t_stat, p_value = scipy_stats.ttest_ind(shared['mean_profit_mad'], indiv['mean_profit_mad'])
metrics["ttest_t_statistic"] = float(t_stat)
metrics["ttest_p_value"] = float(p_value)
metrics["result_significant"] = bool(p_value < 0.05)

logger.info("\n📊 KEY METRICS (all monetary values in MAD):")
logger.info(f"  Total Agents: {metrics['total_agents']}  ({metrics['shared_agents']} SHARED + {metrics['individual_agents']} INDIVIDUAL)")
logger.info(f"  --- YIELD ---")
logger.info(f"  Average Yield (SHARED):     {metrics['avg_yield_shared']:.3f} t/ha")
logger.info(f"  Average Yield (INDIVIDUAL): {metrics['avg_yield_individual']:.3f} t/ha")
logger.info(f"  ⭐ SHARED Yield Advantage:   {metrics['solara_t_ha']:+.3f} t/ha")
logger.info(f"  --- PROFIT (MAD) ---")
logger.info(f"  Average Profit (SHARED):     {metrics['avg_profit_shared_mad']:>12,.0f} MAD")
logger.info(f"  Average Profit (INDIVIDUAL): {metrics['avg_profit_individual_mad']:>12,.0f} MAD")
logger.info(f"  💰 SHARED Profit Advantage:   {metrics['profit_advantage_shared_mad']:>+12,.0f} MAD")
logger.info(f"  --- PROFIT PER HECTARE ---")
logger.info(f"  SHARED:     {metrics['avg_profit_per_ha_shared']:>10,.0f} MAD/ha")
logger.info(f"  INDIVIDUAL: {metrics['avg_profit_per_ha_individual']:>10,.0f} MAD/ha")
logger.info(f"  --- COST BREAKDOWN ---")
logger.info(f"  Avg Revenue SHARED: {metrics['avg_revenue_shared_mad']:>12,.0f} MAD  |  INDIVIDUAL: {metrics['avg_revenue_individual_mad']:>12,.0f} MAD")
logger.info(f"  Avg Cost    SHARED: {metrics['avg_cost_shared_mad']:>12,.0f} MAD  |  INDIVIDUAL: {metrics['avg_cost_individual_mad']:>12,.0f} MAD")
logger.info(f"  --- STATISTICAL TEST ---")
logger.info(f"  Welch's t-test: t={metrics['ttest_t_statistic']:.3f}, p={metrics['ttest_p_value']:.4f}")
sig = "YES ✅" if metrics['result_significant'] else "NO ⚠️"
logger.info(f"  Statistically significant at α=0.05: {sig}")

# ============================================================================
# STATISTICS TABLE
# ============================================================================
print("\n" + "="*60)
print("STATISTICAL SUMMARY BY STRATEGY")
print("="*60)
print(df.groupby('strategy')[['mean_yield', 'mean_profit_mad', 'profit_per_ha_mad',
                               'mean_revenue_mad', 'mean_cost_mad']].describe().round(1))

# ============================================================================
# VISUALIZATIONS
# ============================================================================
cfg_obj = get_config()
viz_cfg = cfg_obj.visualization
fig_width = viz_cfg.get("fig_size_width", 12) if viz_cfg else 12
fig_height = viz_cfg.get("fig_size_height", 6) if viz_cfg else 6
style = viz_cfg.get("plot_style", "seaborn") if viz_cfg else "seaborn"
try:
    plt.style.use(style)
except:
    try:
        plt.style.use("seaborn-v0_8")
    except:
        pass

# --- Figure 1: Strategy Comparison (Yield + Profit) ---
fig, axes = plt.subplots(1, 2, figsize=(fig_width, fig_height))
fig.suptitle('Strategy Comparison: INDIVIDUAL vs SHARED', fontsize=14, fontweight='bold')
if sns is not None:
    sns.boxplot(x='strategy', y='mean_yield', data=df, ax=axes[0], palette="Set2")
    axes[0].set_title('Yield by Strategy')
    axes[0].set_ylabel('Mean Yield (t/ha)')
    sns.boxplot(x='strategy', y='profit_per_ha_mad', data=df, ax=axes[1], palette="Set2")
    axes[1].set_title('Profit per Hectare by Strategy')
    axes[1].set_ylabel('Profit (MAD/ha)')
else:
    df.boxplot(column='mean_yield', by='strategy', ax=axes[0])
    df.boxplot(column='profit_per_ha_mad', by='strategy', ax=axes[1])
plt.tight_layout()
plt.savefig(output_dir / '01_strategy_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

# --- Figure 2: Temporal Dynamics ---
fig, axes = plt.subplots(2, 1, figsize=(fig_width, fig_height * 1.5), sharex=True)
fig.suptitle('Temporal Dynamics over Simulation', fontsize=14, fontweight='bold')

axes[0].plot(ts_df['timestep'], ts_df['avg_yield_shared'], label='SHARED', color='#4ECDC4', linewidth=2)
axes[0].plot(ts_df['timestep'], ts_df['avg_yield_individual'], label='INDIVIDUAL', color='#FF6B6B', linewidth=2)
# Shade drought timesteps
for _, row in ts_df.iterrows():
    if row['drought_zones'] > 0:
        axes[0].axvspan(row['timestep'] - 0.5, row['timestep'] + 0.5, alpha=0.15, color='orange')
    if row['flood_zones'] > 0:
        axes[0].axvspan(row['timestep'] - 0.5, row['timestep'] + 0.5, alpha=0.15, color='blue')
axes[0].set_ylabel('Avg Yield (t/ha)')
axes[0].legend()
axes[0].set_title('Yield per Timestep (orange=drought, blue=flood)')

axes[1].plot(ts_df['timestep'], ts_df['avg_profit_shared'], label='SHARED', color='#4ECDC4', linewidth=2)
axes[1].plot(ts_df['timestep'], ts_df['avg_profit_individual'], label='INDIVIDUAL', color='#FF6B6B', linewidth=2)
for _, row in ts_df.iterrows():
    if row['drought_zones'] > 0:
        axes[1].axvspan(row['timestep'] - 0.5, row['timestep'] + 0.5, alpha=0.15, color='orange')
    if row['flood_zones'] > 0:
        axes[1].axvspan(row['timestep'] - 0.5, row['timestep'] + 0.5, alpha=0.15, color='blue')
axes[1].set_ylabel('Avg Profit (MAD)')
axes[1].set_xlabel('Timestep (Season)')
axes[1].legend()
axes[1].set_title('Profit per Timestep')

plt.tight_layout()
plt.savefig(output_dir / '02_temporal_dynamics.png', dpi=100, bbox_inches='tight')
plt.show()

# --- Figure 3: Revenue vs Cost Breakdown ---
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
x = ['SHARED', 'INDIVIDUAL']
revenues = [metrics['avg_revenue_shared_mad'], metrics['avg_revenue_individual_mad']]
costs = [metrics['avg_cost_shared_mad'], metrics['avg_cost_individual_mad']]
profits = [metrics['avg_profit_shared_mad'], metrics['avg_profit_individual_mad']]

bar_width = 0.25
r1 = np.arange(len(x))
r2 = r1 + bar_width
r3 = r1 + 2 * bar_width

bars1 = ax.bar(r1, revenues, width=bar_width, label='Revenue', color='#4ECDC4')
bars2 = ax.bar(r2, costs, width=bar_width, label='Cost', color='#FF6B6B')
bars3 = ax.bar(r3, profits, width=bar_width, label='Profit', color='#45B7D1')

ax.set_xlabel('Strategy')
ax.set_ylabel('MAD')
ax.set_title('Revenue / Cost / Profit Breakdown by Strategy', fontsize=14, fontweight='bold')
ax.set_xticks(r1 + bar_width)
ax.set_xticklabels(x)
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
plt.tight_layout()
plt.savefig(output_dir / '03_revenue_cost_profit.png', dpi=100, bbox_inches='tight')
plt.show()

# --- Figure 4: Zone Performance ---
fig, axes = plt.subplots(1, 2, figsize=(fig_width, fig_height))
zone_yield = df.groupby(['zone', 'strategy'])['mean_yield'].mean().unstack(fill_value=0)
zone_yield.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4'])
axes[0].set_title('Yield by Zone & Strategy', fontweight='bold')
axes[0].set_ylabel('Mean Yield (t/ha)')
axes[0].tick_params(axis='x', rotation=45)

zone_profit = df.groupby(['zone', 'strategy'])['profit_per_ha_mad'].mean().unstack(fill_value=0)
zone_profit.plot(kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4'])
axes[1].set_title('Profit/ha by Zone & Strategy', fontweight='bold')
axes[1].set_ylabel('Profit (MAD/ha)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

plt.tight_layout()
plt.savefig(output_dir / '04_zone_performance.png', dpi=100, bbox_inches='tight')
plt.show()

# --- Figure 5: Profit Distribution (Violin) ---
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
if sns is not None:
    sns.violinplot(x='strategy', y='profit_per_ha_mad', data=df, ax=ax,
                   palette="muted", inner='quartile')
    ax.set_title('Profit per Hectare Distribution', fontsize=14, fontweight='bold')
    ax.set_ylabel('Profit (MAD/ha)')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
plt.tight_layout()
plt.savefig(output_dir / '05_profit_distribution.png', dpi=100, bbox_inches='tight')
plt.show()

# --- Figure 6: Top crops by final adoption ---
fig, ax = plt.subplots(figsize=(fig_width, fig_height))
crop_counts = df.groupby(['final_crop', 'strategy']).size().unstack(fill_value=0)
top_crops = crop_counts.sum(axis=1).nlargest(10).index
crop_counts_top = crop_counts.loc[top_crops]
crop_counts_top.plot(kind='barh', ax=ax, color=['#FF6B6B', '#4ECDC4'])
ax.set_title('Top 10 Crops by Final Adoption', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Farmers')
ax.legend(title='Strategy')
plt.tight_layout()
plt.savefig(output_dir / '06_crop_adoption.png', dpi=100, bbox_inches='tight')
plt.show()

logger.info("✓ Saved all 6 figures to results/")

# ============================================================================
# SAVE RESULTS
# ============================================================================
# Agent results
df.to_csv(output_dir / "agents_results.csv", index=False, encoding='utf-8')
logger.info(f"✓ Saved: agents_results.csv ({len(df)} agents)")

# Timestep history
ts_df.to_csv(output_dir / "timestep_history.csv", index=False, encoding='utf-8')
logger.info(f"✓ Saved: timestep_history.csv ({len(ts_df)} timesteps)")

# Metrics as JSON
with open(output_dir / "metrics.json", "w", encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
logger.info("✓ Saved: metrics.json")

# Config backup
with open(output_dir / "simulation_config.yaml", "w", encoding='utf-8') as f:
    yaml.dump(config.to_dict(), f)
logger.info("✓ Saved: simulation_config.yaml")

# Summary report
with open(output_dir / "summary_report.txt", "w", encoding="utf-8") as f:
    f.write("AGRICULTURAL AGENT-BASED SIMULATION - SUMMARY REPORT\n")
    f.write("=" * 70 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Experiment: {cfg_obj.experiment.get('name', 'baseline_run')}\n")
    f.write(f"Duration: {elapsed:.2f} seconds\n")
    f.write(f"\n{'='*70}\nCONFIGURATION\n{'='*70}\n")
    f.write(f"Agents: {metrics['total_agents']}  |  Timesteps: {n_timesteps}  |  Seed: {config.simulation.get('random_seed')}\n")
    f.write(f"SHARED price premium: {config.finance.get('shared_price_premium', 0.20):.0%}\n")
    f.write(f"SHARED cost efficiency: {config.finance.get('shared_cost_efficiency', 0.70):.0%} of base cost\n")
    f.write(f"Post-harvest loss: SHARED={config.finance.get('shared_post_harvest_loss', 0.05):.0%}  INDIVIDUAL={config.finance.get('individual_post_harvest_loss', 0.15):.0%}\n")
    f.write(f"Environmental stressors: drought_prob={config.yield_params.get('drought_years_probability', 0.15)}, "
            f"flood_prob={config.yield_params.get('flood_probability', 0.08)}, "
            f"pest_baseline={config.yield_params.get('pest_pressure_baseline', 0.4)}\n")
    f.write(f"\n{'='*70}\nKEY FINDINGS\n{'='*70}\n")
    f.write(f"{'Metric':<35} {'SHARED':>15} {'INDIVIDUAL':>15} {'Advantage':>15}\n")
    f.write("-" * 80 + "\n")
    f.write(f"{'Avg Yield (t/ha)':<35} {metrics['avg_yield_shared']:>15.3f} {metrics['avg_yield_individual']:>15.3f} {metrics['solara_t_ha']:>+15.3f}\n")
    f.write(f"{'Avg Profit (MAD)':<35} {metrics['avg_profit_shared_mad']:>15,.0f} {metrics['avg_profit_individual_mad']:>15,.0f} {metrics['profit_advantage_shared_mad']:>+15,.0f}\n")
    f.write(f"{'Profit/ha (MAD/ha)':<35} {metrics['avg_profit_per_ha_shared']:>15,.0f} {metrics['avg_profit_per_ha_individual']:>15,.0f} {metrics['avg_profit_per_ha_shared'] - metrics['avg_profit_per_ha_individual']:>+15,.0f}\n")
    f.write(f"{'Avg Revenue (MAD)':<35} {metrics['avg_revenue_shared_mad']:>15,.0f} {metrics['avg_revenue_individual_mad']:>15,.0f} {metrics['avg_revenue_shared_mad'] - metrics['avg_revenue_individual_mad']:>+15,.0f}\n")
    f.write(f"{'Avg Cost (MAD)':<35} {metrics['avg_cost_shared_mad']:>15,.0f} {metrics['avg_cost_individual_mad']:>15,.0f} {metrics['avg_cost_shared_mad'] - metrics['avg_cost_individual_mad']:>+15,.0f}\n")
    f.write(f"\n{'='*70}\nSTATISTICAL SIGNIFICANCE\n{'='*70}\n")
    f.write(f"Welch's t-test: t={metrics['ttest_t_statistic']:.3f}, p={metrics['ttest_p_value']:.4f}\n")
    f.write(f"Significant at α=0.05: {'YES' if metrics['result_significant'] else 'NO'}\n")
    f.write(f"\n{'='*70}\nDATA SOURCES\n{'='*70}\n")
    f.write("Market prices: FAOSTAT Producer Prices - Morocco (CC BY-4.0)\n")
    f.write("Labor costs: Morocco SMAG 2024 (Decree 2.23.993): 84.37 MAD/day\n")
    f.write("Water tariffs: ORMVA official irrigation tariffs\n")
    f.write("Fertilizer: OCP Group domestic prices\n")
    f.write("Ecological params: FAO EcoCrop Database\n")
    f.write("Yield baselines: FAOSTAT crop production statistics (2018-2022 averages)\n")
logger.info("✓ Saved: summary_report.txt")

2026-02-14 20:09:08,271 - agroai_sim - INFO - 
2026-02-14 20:09:08,271 - agroai_sim - INFO - ANALYSIS & RESULTS
2026-02-14 20:09:08,277 - agroai_sim - INFO - ============================================================
2026-02-14 20:09:08,363 - agroai_sim - INFO - 
📊 KEY METRICS (all monetary values in MAD):
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4ca' in position 48: character maps to <undefined>
Call stack:
  File "<frozen runpy>", l

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u03b1' in position 77: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users


STATISTICAL SUMMARY BY STRATEGY
           mean_yield                                      mean_profit_mad  \
                count mean  std  min  25%  50%   75%   max           count   
strategy                                                                     
INDIVIDUAL      100.0  8.0  9.6  0.9  1.1  1.2  21.0  22.5           100.0   
SHARED          100.0  8.0  9.6  0.9  1.1  1.2  21.0  22.5           100.0   

                      ... mean_revenue_mad           mean_cost_mad           \
                mean  ...              75%       max         count     mean   
strategy              ...                                                     
INDIVIDUAL   86812.4  ...         181358.3  704204.4         100.0  47048.9   
SHARED      141793.1  ...         257467.7  888427.3         100.0  36593.6   

                                                                  
                std     min      25%      50%      75%       max  
strategy                                      

C:\Users\pc\AppData\Local\Temp\ipykernel_24312\825052127.py:147: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='strategy', y='mean_yield', data=df, ax=axes[0], palette="Set2")
C:\Users\pc\AppData\Local\Temp\ipykernel_24312\825052127.py:150: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='strategy', y='profit_per_ha_mad', data=df, ax=axes[1], palette="Set2")
C:\Users\pc\AppData\Local\Temp\ipykernel_24312\825052127.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\pc\AppData\Local\Temp\ipykernel_24312\825052127.py:190: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\pc\AppData\Local\Temp\

In [20]:
# ============================================================================
# 🌐 LAUNCH INTERACTIVE SOLARA DASHBOARD
# ============================================================================
logger.info("\n" + "="*60)
logger.info("🚀 LAUNCH INTERACTIVE DASHBOARD")
logger.info("="*60)

try:
    import subprocess
    import webbrowser
    from pathlib import Path
    
    dashboard_file = Path("dashboard_solara.py")
    
    if dashboard_file.exists():
        logger.info("✓ Solara dashboard file found")
        logger.info("📡 Starting Solara server...")
        logger.info("🌐 Dashboard will be available at: http://localhost:8000")
        logger.info("")
        logger.info("💡 TIP: The dashboard will auto-load your simulation results!")
        logger.info("    - Interactive plots with filtering")
        logger.info("    - Performance metrics overview") 
        logger.info("    - Zone analysis and distributions")
        logger.info("    - Raw data explorer")
        logger.info("")
        logger.info("⚠️  To start the dashboard, run in terminal:")
        logger.info("   solara run dashboard_solara.py")
        logger.info("")
        logger.info("🔄 Or uncomment the lines below to auto-launch:")
        
        # Uncomment these lines to auto-launch the dashboard
        # logger.info("🚀 Auto-launching dashboard...")
        # subprocess.Popen(["solara", "run", "dashboard_solara.py"], cwd=Path.cwd())
        # import time
        # time.sleep(3)  # Wait for server to start
        # webbrowser.open("http://localhost:8000")
        # logger.info("✅ Dashboard launched! Check your browser.")
        
    else:
        logger.warning("❌ dashboard_solara.py not found")
        logger.info("💡 The dashboard file should be in the project root directory")
        
except ImportError as e:
    logger.error(f"❌ Missing required package: {e}")
    logger.info("💡 Install required packages:")
    logger.info("   pip install solara solara-plotly")
except Exception as e:
    logger.error(f"❌ Error launching dashboard: {e}")

logger.info("\n" + "="*60)
logger.info("🎊 SIMULATION COMPLETE - READY FOR ANALYSIS!")
logger.info("="*60)
logger.info("")
logger.info("📊 Results Summary:")
logger.info(f"   • Total Farmers: {metrics['total_agents']}")
logger.info(f"   • SHARED Strategy: {metrics['shared_agents']} farmers ({metrics['shared_agents']/metrics['total_agents']*100:.1f}%)")
logger.info(f"   • Average Yield: {metrics['avg_yield_all']:.3f} t/ha")
logger.info(f"   • SHARED Advantage: {metrics['solara_t_ha']:.3f} t/ha")
logger.info("")
logger.info("📁 Generated Files:")
logger.info("   • Static plots: results/*.png") 
logger.info("   • Agent data: results/agents_results.csv")
logger.info("   • Metrics: results/metrics.json")
logger.info("   • Config backup: results/simulation_config.yaml")
logger.info("   • Summary report: results/summary_report.txt")
logger.info("")
logger.info("🌐 Interactive Dashboard:")
logger.info("   • File: dashboard_solara.py")
logger.info("   • Launch: solara run dashboard_solara.py") 
logger.info("   • Access: http://localhost:8000")
logger.info("")
logger.info("🎯 Next Steps:")
logger.info("   1. Review the static plots above")
logger.info("   2. Launch the Solara dashboard for interactive exploration")
logger.info("   3. Modify config.yaml for different experiments")
logger.info("   4. Re-run this notebook to compare results")
logger.info("")
logger.info("=" * 60)

2026-02-14 20:09:12,337 - agroai_sim - INFO - 
2026-02-14 20:09:12,340 - agroai_sim - INFO - 🚀 LAUNCH INTERACTIVE DASHBOARD
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f680' in position 46: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\lo